# ReSProp Training Harness

Operation based on implementation in https://github.com/negargoli/ReSprop

Paper : [ReSprop: Reuse Sparsified Backpropagation](https://openaccess.thecvf.com/content_CVPR_2020/papers/Goli_ReSprop_Reuse_Sparsified_Backpropagation_CVPR_2020_paper.pdf) by Negar Goli, Tor M. Aamodt

To run, please first extract the zip file attached with the code (ResProp_with_fix.zip)


In [ ]:
!pip install torch==1.1 torchvision==0.3

In [ ]:
!cd ReSprop/backward
## For colab
#!CPATH=/usr/local/cuda-10.0/include python setup.py install
!python setup.py install
!cd /content/ReSprop

In [ ]:
# Standard Python Import
import time
from PIL import Image
import datetime
import os

# Python based libraries
import tqdm

# torch
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, sampler
import torch.optim as optim
from tensorboardX import SummaryWriter

# torchvision
from torchvision.datasets import CIFAR10
import torchvision.transforms as T

# ResProp import
from models.resnet_ImageNet import resnet18

def seed_everything(seed: int):
  import random, os
  import numpy as np
  import torch
  random.seed(seed)
  os.environ['PYTHONHASHSEED'] = str(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = True

seed_everything(9999)


In [ ]:
device = torch.device('cuda')


transform_train = T.Compose([
    T.RandomCrop(32, padding=4),
    T.RandomHorizontalFlip(),
    T.ToTensor(),
    T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = T.Compose([
    T.ToTensor(),
    T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

class ChunkSampler(sampler.Sampler):
    """Samples elements sequentially from some offset. 
    Arguments:
        num_samples: # of desired datapoints
        start: offset where we should start selecting from
    """
    def __init__(self, num_samples, start = 0):
        self.num_samples = num_samples
        self.start = start

    def __iter__(self):
        return iter(range(self.start, self.start + self.num_samples))

    def __len__(self):
        return self.num_samples


batch_size = 128

trainset = CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_train)
valset = CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_test)

trainloader = DataLoader(trainset, batch_size=batch_size, num_workers=2, drop_last=True, sampler=ChunkSampler(40000, 0))
val_loader = DataLoader(valset, batch_size=batch_size, num_workers=2, drop_last=True, sampler=ChunkSampler(10000, 40000))

testset = CIFAR10(root='./data', train=False,
                                       download=True, transform=transform_test)

testloader = DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

In [ ]:
torch.cuda.empty_cache()
criterion = nn.NLLLoss()
optimizers = ['Adagrad']
device = torch.device("cuda")
for optim in optimizers:
    best_val_acc = -99999
    model = resnet18(391, 128, 0, 20, 1).to(device)
    model.fc = nn.Sequential(nn.Linear(512, 10), nn.LogSoftmax(dim=1))
    model.to(device)
    d = datetime.datetime.now().strftime('%Y-%m-%d~%H:%M:%S')
    path = f'/content/drive/MyDrive/Lemma/approx/{optim}/{d}'
    print(f'Saving model at {path}')
    if not os.path.exists(path + '/ckpt'):
        os.makedirs(path + '/ckpt')
        os.makedirs(path + '/log')
    save_tbx_log = path + '/log'
    writer = SummaryWriter(save_tbx_log)    
    save_tbx_log = path + '/log'
    print(f'Working on optimizer {optim}')
    
    if optim == 'SGD':
      optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)
    elif optim == 'SGD_mom':
      optimizer = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.99, nesterov=True)
    if optim == 'Adam':
      optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
    elif optim == 'Adadelta':
      optimizer = torch.optim.Adadelta(model.parameters(), lr=1e-2)
    elif optim == 'RMSProp':
      optimizer = torch.optim.RMSprop(model.parameters(), lr=1e-2)
    elif optim == 'Adagrad':
      optimizer = torch.optim.Adagrad(model.parameters(), lr=1e-2)
    
    start_time = time.time()
    for epoch in range(40):
        model.train()
        running_loss      = 0.0
        running_val_loss  = 0.0
        train_correct     = 0
        val_correct       = 0
        for data in tqdm.tqdm(trainloader):
            
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device) 
            
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            pred = outputs.max(1, keepdim=True)[1]
            train_correct += pred.eq(labels.view_as(pred)).sum().item()
            
        writer.add_scalar('Train/train_loss', running_loss / len(trainloader), epoch)
        writer.add_scalar('Train/train_acc', (train_correct / 40000) * 100, epoch)
        with torch.no_grad():
          model.eval()
          for data in tqdm.tqdm(val_loader):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            running_val_loss += loss.item()

            pred = outputs.max(1, keepdim=True)[1]
            val_correct += pred.eq(labels.view_as(pred)).sum().item()
        val_acc = (val_correct / 10000) * 100
        if val_acc > best_val_acc:
          best_val_acc = val_acc
          print(f'best_val_acc:, {best_val_acc:3f}')
          state = {
                    'epoch': epoch,
                    'best_val_acc': best_val_acc,
                    'state_dict': model.state_dict(),
                    'optimizer': optimizer.state_dict(),
                    }
          torch.save(state, f'{path}/ckpt/{d}.pth')
        writer.add_scalar('Validation/val_loss', running_val_loss / len(val_loader), epoch)
        writer.add_scalar('Validation/val_acc', (val_correct / 10000) * 100, epoch) 
        
        
        print(f'[{epoch + 1}] train_loss: {running_loss / len(trainloader):.3f}, train_acc: {(train_correct / 40000) * 100:.3f} val_loss: {running_val_loss / len(val_loader):.3f}, val_acc: {(val_correct / 10000) * 100:.3f}')
        
        train_correct     = 0
        val_correct       = 0
        running_val_loss  = 0.0
        running_loss = 0.0
    end_time = time.time()

    print(f'Time Taken {end_time - start_time} sec')

    running_test_loss = 0.0
    test_correct = 0
    with torch.no_grad():
      model.eval()
      for data in tqdm.tqdm(testloader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        running_test_loss += loss.item()
        
        pred = outputs.max(1, keepdim=True)[1]
        test_correct += pred.eq(labels.view_as(pred)).sum().item()
    
    writer.add_scalar('Test/test_loss', running_test_loss / len(testloader), 1)
    writer.add_scalar('Test/test_acc', (test_correct / len(testset)) * 100, 1)

    print(f'For Optimizer {optim} test_loss: {running_test_loss / len(testloader):.3f}, test_acc: {(test_correct / len(testset)) * 100:.3f}')


print('Finished Training')